In [113]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv('C:/Users/pjjj3/Downloads/dataset.csv')

In [3]:
data

,期間,NTD/USD,Consumer sentiment,federal fund rate change rate,10 year treasury bond rate minus 2 year,30 year treasury bond rate change rate,Real M2 Money Stock growth rate,Industrial Production Manufacturing (NAICS) growth rate,cpi growth rate,Capacity Utilization,New One Family Houses Sold,unemployment rate,NBER based Recession Indicators,台灣領先指標不含趨勢指數(點),台灣同時指標不含趨勢指數(點),EAESI
0,1982M01,37.712,71.0,0.068715,0.02,0.057249,0.005253,-0.024147,0.003226,75.0489,28,8.6,1,100.14,106.65,69.7
1,1982M02,37.912,66.5,0.118003,-0.39,0.000000,-0.000853,0.027916,0.005359,76.3768,29,8.9,1,99.73,103.95,67.1
2,1982M03,38.097,62.0,-0.006766,-0.33,-0.048523,0.006778,-0.008731,0.001066,75.6450,36,9.0,1,99.24,101.63,66.8
3,1982M04,38.337,65.5,0.017711,-0.33,-0.011826,0.006520,-0.006654,0.008520,74.8423,32,9.3,1,98.46,99.66,67.8
4,1982M05,38.756,67.5,-0.032798,-0.16,-0.009723,-0.003055,-0.001994,0.007392,74.2336,36,9.4,1,97.48,97.99,66.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,2020M11,28.844,76.9,0.000000,0.70,0.031847,0.012634,0.010363,0.001741,73.9102,61,6.7,1,104.29,102.20,89.3
467,2020M12,28.539,80.7,0.000000,0.79,0.030864,0.001613,0.008261,0.000448,74.6682,63,6.7,1,104.76,103.20,92.4
468,2021M01,28.431,79.0,0.000000,0.95,0.089820,0.009114,0.013098,0.000315,75.3393,77,6.3,1,105.05,104.17,91.5
469,2021M02,28.350,76.8,-0.111111,1.14,0.120879,0.010559,-0.037246,0.001015,73.3560,70,6.2,1,105.23,105.10,93.4


In [4]:
X = data.iloc[:, 2:].values
usd = data.iloc[:, 1].values
usd = np.append(usd, 27.86409)

In [5]:
y = []
for i in range(1, len(usd)):
    if usd[i] > usd[i-1]:
        y.append(1)
    else:
        y.append(0)

In [6]:
y = np.array(y)

In [7]:
len(y), len(X)

(471, 471)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

In [172]:
forest = RandomForestClassifier(criterion='gini',
                                n_estimators=6000, 
                                random_state=1,
                                n_jobs=-1)
forest.fit(X_train, y_train)

RandomForestClassifier(n_estimators=6000, n_jobs=-1, random_state=1)

In [173]:
scores = cross_val_score(estimator=forest,
                             X=X_test,
                             y=y_test,
                             cv=10
                             )

In [174]:
scores

array([0.53333333, 0.53333333, 0.64285714, 0.71428571, 0.64285714,
       0.64285714, 0.5       , 0.5       , 0.85714286, 0.42857143])

In [175]:
scores.mean(), scores.std()

(0.5995238095238096, 0.11956933301154354)

In [188]:
svm = SVC(kernel='rbf', gamma=0.07, random_state=1, C=100)
svm.fit(X_train, y_train)

SVC(C=100, gamma=0.07, random_state=1)

In [189]:
scores2 = cross_val_score(estimator=svm,
                          X=X_test,
                          y=y_test,
                          cv=10)

In [190]:
scores2

array([0.66666667, 0.73333333, 0.64285714, 0.64285714, 0.85714286,
       0.57142857, 0.64285714, 0.57142857, 0.5       , 0.57142857])

In [191]:
scores2.mean(), scores2.std()

(0.64, 0.09502834044393463)

In [220]:
lr = LogisticRegression(C=15, random_state=1, solver='lbfgs', multi_class='multinomial', max_iter=10000)
lr.fit(X_train, y_train)

LogisticRegression(C=15, max_iter=10000, multi_class='multinomial',
                   random_state=1)

In [221]:
scores3 = cross_val_score(estimator=lr,
                          X=X_test,
                          y=y_test,
                          cv=10)

In [222]:
scores3

array([0.46666667, 0.66666667, 0.71428571, 0.5       , 0.5       ,
       0.5       , 0.64285714, 0.57142857, 0.5       , 0.64285714])

In [223]:
scores3.mean(), scores3.std()

(0.5704761904761905, 0.08429512995565262)

In [247]:
knn = KNeighborsClassifier(n_neighbors=20, p=2, metric='minkowski')
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=20)

In [248]:
scores4 = cross_val_score(estimator=knn,
                          X=X_test,
                          y=y_test,
                          cv=10)

In [249]:
scores4

array([0.6       , 0.53333333, 0.57142857, 0.64285714, 0.71428571,
       0.64285714, 0.42857143, 0.5       , 0.57142857, 0.64285714])

In [250]:
scores4.mean(), scores4.std()

(0.5847619047619047, 0.07855844048495728)

In [107]:
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.pipeline import _name_estimators
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder

In [317]:
class MajorityVoteClassifier(BaseEstimator, 
                             ClassifierMixin):
    """ A majority vote ensemble classifier

    Parameters
    ----------
    classifiers : array-like, shape = [n_classifiers]
      Different classifiers for the ensemble

    vote : str, {'classlabel', 'probability'} (default='classlabel')
      If 'classlabel' the prediction is based on the argmax of
        class labels. Else if 'probability', the argmax of
        the sum of probabilities is used to predict the class label
        (recommended for calibrated classifiers).

    weights : array-like, shape = [n_classifiers], optional (default=None)
      If a list of `int` or `float` values are provided, the classifiers
      are weighted by importance; Uses uniform weights if `weights=None`.

    """
    def __init__(self, classifiers, vote='classlabel', weights=None):

        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value
                                  in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights

    def fit(self, X, y):
        """ Fit classifiers.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_examples, n_features]
            Matrix of training examples.

        y : array-like, shape = [n_examples]
            Vector of target class labels.

        Returns
        -------
        self : object

        """
        if self.vote not in ('probability', 'classlabel'):
            raise ValueError("vote must be 'probability' or 'classlabel'"
                             "; got (vote=%r)"
                             % self.vote)

        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d classifiers'
                             % (len(self.weights), len(self.classifiers)))

        # Use LabelEncoder to ensure class labels start with 0, which
        # is important for np.argmax call in self.predict
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self

    def predict(self, X):
        """ Predict class labels for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_examples, n_features]
            Matrix of training examples.

        Returns
        ----------
        maj_vote : array-like, shape = [n_examples]
            Predicted class labels.
            
        """
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1)
        else:  # 'classlabel' vote

            #  Collect results from clf.predict calls
            predictions = np.asarray([clf.predict(X)
                                      for clf in self.classifiers_]).T

            maj_vote = np.apply_along_axis(
                                      lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions)
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote

    def predict_proba(self, X):
        """ Predict class probabilities for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_examples, n_features]
            Training vectors, where n_examples is the number of examples and
            n_features is the number of features.

        Returns
        ----------
        avg_proba : array-like, shape = [n_examples, n_classes]
            Weighted average probability for each class per example.

        """
        probas = np.asarray([clf.predict_proba(X)
                             for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba

    def get_params(self, deep=True):
        """ Get classifier parameter names for GridSearch"""
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in self.named_classifiers.items():
                for key, value in step.get_params(deep=True).items():
                    out['%s__%s' % (name, key)] = value
            return out

In [319]:
mv_clf = MajorityVoteClassifier(classifiers=[forest, svm, knn])

In [268]:
scores5 = cross_val_score(estimator=mv_clf,
                             X=X_test,
                             y=y_test,
                             cv=10)

In [269]:
scores5

array([0.66666667, 0.6       , 0.64285714, 0.64285714, 0.78571429,
       0.64285714, 0.64285714, 0.57142857, 0.64285714, 0.57142857])

In [270]:
scores5.mean(), scores5.std()

(0.6409523809523809, 0.05742786069211938)

In [333]:
#use 2021/3 to try predicting
mv_clf.fit(X_train, y_train)
xx = np.array([84.9, -0.125, 1.46, 0.147059, 0.005285, 0.027859, 0.003385, 74.3914, 97 ,6, 1, 105.35, 105.98, 100.9])
xx = xx.reshape(1, -1)

In [334]:
mv_clf.predict(xx)

array([0])